In [4]:
import numpy as np
from ipywidgets import FloatSlider, SelectionSlider

import plotlymath as pm
from myutils import interact, ddeint

In [88]:
pm.set_defaults(margins=(20, 20, 40, 40))

In [89]:
def euler_interactive(vector_field, initial_state, step_sizes, max_steps, partial=False):
    figure, plot = pm.make_figure(widget=True)
    plot.axes_labels(r"$x$", r"$y$")
    plot.axes_ranges((-2, 2), (-2, 2), scale=(1, 1))
    plot.vector_field(lambda x, y: vector_field(0, (x, y)), (-2, 2), (-2, 2), 
            color="limegreen", plotpoints=13, name="Vector field", legendrank=1)

    iteration_list = np.arange(max_steps + 1)
    if partial:
        iteration_list = (iteration_list.reshape(-1, 1) + (0.0, 0.1, 0.2)).flatten()[:-2]

    last_step_size = None
    state_list = []
    @interact(iterations=SelectionSlider(options=iteration_list, description="# of steps"), 
              step_size=SelectionSlider(options=step_sizes, description="Step size"))
    def update(iterations, step_size):
        nonlocal last_step_size, state_list
        if step_size != last_step_size:
            last_step_size = step_size
            state = np.array(initial_state, dtype=float)
            state_list = [state]
            for i in range(max_steps):
                t = i * step_size
                state = state + step_size * vector_field(t, state_list[-1])
                state_list.append(state)
        iterations, substep = divmod(round(10*iterations), 10)
        vector = vector_field(iterations * step_size, state_list[iterations])
        color = "green" if substep == 1 else "lightgreen"
        visible = True if substep else "legendonly" if partial else False
        with figure.batch_update():
            plot.vector(vector, start=state_list[iterations], color=color, line_width=5, 
                    visible=visible, legendrank=1001, name="Current vector", id="vector")
            iterations += 2 if substep == 2 else 1
            plot.lines(state_list[:iterations], color="blue", mode="markers+lines", 
                    marker_color="black", name="Simulation", id="euler")
            plot.points((initial_state,), color="black", size=10, legendrank=2, 
                    name="Initial state", id="init")

    return figure

# Euler's method

The interactive below allows you to play with Euler's method, with various step sizes. Here we are simulating the differential equation 
$$ \begin{cases} x' = y \\ y' = -x \end{cases} $$
or, in matrix form, 
$$ \begin{bmatrix} x' \\ y' \end{bmatrix} = \begin{pmatrix} 0 & 1 \\ -1 & 0 \end{pmatrix} \begin{bmatrix} x \\ y \end{bmatrix} $$

With the initial condition $x(0) = 1, y(0) = 0$, this differential equation has the explicit solution 
$$ \begin{cases} x(t) = \cos(t) \\ y(t) = -\sin(t) \end{cases} $$
which describes a circle (traversed clockwise) in the $xy$-plane. 


In [90]:
def vector_field(t, state):
    x, y = state
    return np.array((y, -x), dtype=float)

euler_interactive(vector_field, (1, 0), (1, 0.5, 0.3, 0.2, 0.1, 0.05), 150, partial=True)

interactive(children=(SelectionSlider(description='# of steps', options=(0.0, 0.1, 0.2, 1.0, 1.1, 1.2, 2.0, 2.…

FigureWidget({
    'data': [{'fill': 'toself',
              'fillcolor': 'limegreen',
              'legendra…

In [91]:
euler_interactive(vector_field, (1, 0), (1, 0.5, 0.3, 0.2, 0.1, 0.05, 0.02, 0.01), 628)

interactive(children=(SelectionSlider(description='# of steps', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,…

FigureWidget({
    'data': [{'fill': 'toself',
              'fillcolor': 'limegreen',
              'legendra…

# An example of a delay differential equation

Below is the time series of a simulation of a delay differential equation (DDE). Specifically, this is a simulation of the *Mackey–Glass* equation: 
$$ x'(t) = 2 - 6 x(t) \cdot \frac{x(t - 1.8)^2}{1 + x(t - 1.8)^2} $$

Note the time delay of 1.8 within the sigmoid function in the above equation: that sigmoid function is a function of *the value of $x$ 1.8 time units **before** time $t$*. 


In [97]:
def f(t, x):
    return 2 - 6*x(t) * x(t - 1.8)**2 / (1 + x(t - 1.8)**2)

solution = ddeint(f, lambda t: 1, 200, max_step=0.1)
figure, plot = pm.make_figure()
plot.axes_labels("$t$", "$x$")
plot.axes_ranges((0, 50), (0, 1.5))
plot.function(solution, (0, 200), smooth=True, aspect_ratio=40)
figure

In the interactive below, you can see the effect of manipulating the time delay in the above DDE. So here, the DDE is 
$$ x'(t) = 2 - 6 x(t) \cdot \frac{x(t - \tau)^2}{1 + x(t - \tau)^2} $$
where $\tau$, the time delay, is a parameter. (In the subject of DDEs, the symbol $\tau$, the lowercase Greek letter tau, is often used to represent a time delay.) 

Notice that for low values of the time delay, the time series shows no long-term oscillations, but for higher time delays, the long-term behavior is oscillatory. 

This shows a really significant difference between ODEs and DDEs: a single-variable (autonomous) ODE cannot have oscillatory behavior, whereas a single-variable DDE such as this one can. 


In [96]:
def mackey_glass_interactive():
    figure, plot = pm.make_figure(widget=True)
    plot.axes_labels("$t$", "$x$")
    plot.axes_ranges((0, 100), (0, 2))

    @interact(tau=FloatSlider(min=0, max=3, description="Time delay"))
    def update(tau):
        def f(t, x):
            return 2 - 6*x(t) * x(t - tau)**2 / (1 + x(t - tau)**2)

        max_step = 0.1 if tau == 0.0 or tau >= 0.2 else tau/2
        solution = ddeint(f, lambda t: 1, 200, max_step=max_step)
        plot.function(solution, (0, 200), smooth=True, aspect_ratio=36, id="solution")

    return figure

mackey_glass_interactive()

interactive(children=(FloatSlider(value=0.0, description='Time delay', max=3.0), Output()), _dom_classes=('wid…

FigureWidget({
    'data': [{'line': {'shape': 'spline', 'smoothing': 1.3},
              'mode': 'lines',
   …